In [1]:
# Simulation Parameters
T = 100  # Time span for one slot 100ms
mu = 0.02   # Sensing duration ratio 
P1 = 0.5    # Probability of spectrum occupancy
Pd = 0.9    # Probability of detection
Pf = 0.1    # Probability of false alarm
m = 20      # Battery capacity
Eh = 0.1    # Harvested energy during one slot 
Pw = -60    # Primary signal power in dBm
Nw = -70    # Noise power in dBm
g = 10**-5
